<a href="https://colab.research.google.com/github/41371108H/114-1repo/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q install gradio google-generativeai openpyxl


In [4]:
import io, re, os, urllib.request
from urllib.error import HTTPError, URLError
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import gradio as gr

# ================= 工具函式 =================
def _extract_sheet_id_and_gid(sheet_url: str):
    try:
        sheet_id = sheet_url.split("/d/")[1].split("/")[0]
    except Exception:
        raise ValueError("不是有效的 Google Sheet 連結。")
    m = re.search(r"[#&?]gid=(\d+)", sheet_url)
    url_gid = m.group(1) if m else None
    return sheet_id, url_gid

def _download(url: str) -> bytes:
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=30) as r:
        return r.read()

def load_sheet(sheet_url: str, gid: str | None = None) -> pd.DataFrame:
    """穩健讀取 Google Sheet，支援自動 gid 偵測與 xlsx 備援。"""
    sheet_id, url_gid = _extract_sheet_id_and_gid(sheet_url)
    gid_final = gid or url_gid or "0"
    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid_final}"
    try:
        data = _download(csv_url)
        df = pd.read_csv(io.BytesIO(data))
    except Exception:
        xlsx_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx"
        data = _download(xlsx_url)
        df = pd.read_excel(io.BytesIO(data), sheet_name=0)
    df.columns = [str(c).strip() for c in df.columns]
    need = ["星期","節次","上課時間","課程名稱","教室"]
    miss = [c for c in need if c not in df.columns]
    if miss:
        raise ValueError(f"試算表缺少欄位：{', '.join(miss)}")
    for c in need + ["備註"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    return df

# ================= 日期與排序 =================
weekday_map = ["一","二","三","四","五","六","日"]
zh_to_idx = {z:i for i,z in enumerate(weekday_map)}
period_order = [str(i) for i in range(1,21)] + list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
order_map = {p:i for i,p in enumerate(period_order)}

def today_zh() -> str:
    return weekday_map[datetime.now(ZoneInfo("Asia/Taipei")).weekday()]

def normalize_day(x:str)->str:
    m = {"Mon":"一","Tue":"二","Wed":"三","Thu":"四","Fri":"五","Sat":"六","Sun":"日",
         "星期一":"一","星期二":"二","星期三":"三","星期四":"四","星期五":"五","星期六":"六","星期日":"日",
         "週一":"一","週二":"二","週三":"三","週四":"四","週五":"五","週六":"六","週日":"日"}
    return m.get(x.strip(),x.strip())

def filter_day(df, day:str|None):
    target = normalize_day(day) if day else today_zh()
    sub = df[df["星期"].eq(target)].copy()
    if "節次" in sub.columns:
        sub["__o__"] = sub["節次"].map(order_map).fillna(999).astype(int)
        sub = sub.sort_values(["__o__","上課時間"]).drop(columns="__o__",errors="ignore")
    return sub[["星期","節次","上課時間","課程名稱","教室"]]

def filter_week(df):
    days=set(weekday_map)
    sub=df[df["星期"].isin(days)].copy()
    sub["__w__"]=sub["星期"].map(zh_to_idx)
    sub["__o__"]=sub["節次"].map(order_map).fillna(999)
    sub=sub.sort_values(["__w__","__o__","上課時間"])
    return sub[["星期","節次","上課時間","課程名稱","教室"]]

# ================= 規則版提醒 =================
def rule_hint(row):
    n=row.get("課程名稱",""); r=row.get("教室","")
    h="攜帶文具與學生證；提前5分鐘到教室。"
    if any(k in n for k in ["線性代數","微積分","數學"]): h="帶計算機與課本；預習例題。"
    if "機率" in n: h="帶計算機；複習分配與範例題。"
    if any(k in n for k in ["程式","作業系統","網際網路"]): h="帶筆電與充電器；更新環境。"
    if "英文" in n: h="帶課本與單字本；先讀本週文章。"
    if "日語" in n: h="帶課本；預習50音。"
    if "人類與自然資源" in n: h="閱讀案例；思考永續議題。"
    if "體育" in n or "網球" in n or "網球場" in r: h="穿運動服鞋；攜水與毛巾熱身。"
    return h

# ================= Gemini 生成提醒 =================
def gemini_reminders(rows, model, key):
    try:
        import google.generativeai as genai
        genai.configure(api_key=key)
        m=genai.GenerativeModel(model)
        lines=[f"- {r['星期']}節{r['節次']}《{r['課程名稱']}》@{r['教室']}" for r in rows]
        p=("請為以下課程生成一行繁體中文『行前提醒』，聚焦攜帶品或預習章節：\n"+
           "\n".join(lines))
        t=(m.generate_content(p).text or "").strip()
        return [s.strip(" -•") for s in t.splitlines() if s.strip()]
    except Exception:
        return None

def week_reminders(df,use_gem,key,model):
    sub=filter_week(df)
    if sub.empty: return sub
    hints=[rule_hint(r) for _,r in sub.iterrows()]
    if use_gem and key:
        out=gemini_reminders(sub.to_dict("records"),model,key)
        if out and len(out)==len(sub): hints=out
    sub["行前提醒"]=hints
    return sub

# ================= Gradio 介面 =================
def ui_day(url, gid, day):
    try:
        df=load_sheet(url,gid)
        res=filter_day(df,day)
        if res.empty: return gr.update(value=pd.DataFrame()),"⚠️ 無課程"
        return res[["課程名稱","上課時間","教室"]],"✅ 查詢完成"
    except Exception as e:
        return gr.update(value=pd.DataFrame()),f"❌ 錯誤：{e}"

def ui_week(url,gid,use_gem,model,key):
    try:
        df=load_sheet(url,gid)
        res=week_reminders(df,use_gem,key,model)
        if res.empty: return gr.update(value=pd.DataFrame()),"⚠️ 本週無課程"
        return res,"✅ 生成完成"
    except Exception as e:
        return gr.update(value=pd.DataFrame()),f"❌ 錯誤：{e}"

with gr.Blocks(title="課表查詢 + AI 行前提醒") as app:
    gr.Markdown("## 📚 課表查詢 + 🤖 行前提醒")
    url=gr.Textbox(label="Google Sheet 連結",value="https://docs.google.com/spreadsheets/d/14stNRxIhsYQyhIjSxe8OshUovhPdCw9WwADAefsiRsg/edit?usp=sharing")
    gid=gr.Textbox(label="工作表 gid (可留空，自動偵測)",value="")
    with gr.Tab("查某一天"):
        day=gr.Dropdown(["","一","二","三","四","五","六","日"],label="星期（空白=今天）")
        btn1=gr.Button("查詢")
        out1=gr.Dataframe(label="當日課程")
        msg1=gr.Markdown()
        btn1.click(ui_day,[url,gid,day],[out1,msg1])
    with gr.Tab("本週行前提醒（AI 點子）"):
        use_gem=gr.Checkbox(label="使用 Gemini 生成提醒",value=False)
        model=gr.Dropdown(["gemini-1.5-flash","gemini-1.5-pro"],label="Gemini 模型",value="gemini-1.5-flash")
        key=gr.Textbox(label="GEMINI_API_KEY（可留空）",type="password")
        btn2=gr.Button("生成提醒")
        out2=gr.Dataframe(label="本週課程 + 行前提醒")
        msg2=gr.Markdown()
        btn2.click(ui_week,[url,gid,use_gem,model,key],[out2,msg2])

app.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>